In [1]:
import requests
import bs4
import os
import pandas as pd

In [2]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [3]:
def record_date_time(z):
    x=str.split(z,',')
    date =x[0]
    time= x[1] +':00'

    return date,time

def record_URL(x):
    y='https://timesofindia.indiatimes.com'
    url = y+x 
    return url

In [4]:
link ='https://timesofindia.indiatimes.com/city'
res = requests.get(link)
soup =bs4.BeautifulSoup(res.text,'lxml')

In [5]:
List = soup.findAll('div',{'class':'news_card'})

In [14]:
outname ='NDTV_cities-'+str(Today_date)+'.csv'

root = 'CSV/'
if not os.path.exists(root):
    os.mkdir(root)
date_today= Today_date +'/'
outdir=root+ date_today[:-1]
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)

In [15]:
ndtv = pd.read_csv(fullname)

In [17]:
cat = ndtv['Category'].unique()

In [18]:
records = []
for link in List:
    try:
        category =link.find('a')['title'].lower()+' news'
        if category not in cat:
            continue
        url = record_URL(link.find('a')['href'])
        res = requests.get(url)
        soup =bs4.BeautifulSoup(res.text,'lxml')
        main_content = soup.select('.main-content')[0].findAll('li')

        for record in main_content[:25]:
            try:
                media_content = record.find('span',{'class':'w_tle'})

                Heading = media_content.text
                URL= record_URL(media_content.find('a')['href'])

                Date = Today_date
                Time = Today_time
                Category = category
                Source = 'Times of India'
            except:
                continue
            records.append((Source,Heading,Category,Date,Time,URL))
    except:
        continue

In [19]:
import pandas as pd
df=pd.DataFrame(records,columns=['Source','Heading','Category','Date','Time','URL'])
df['Date'] = pd.to_datetime(df['Date'])

In [21]:
df.head()

,Source,Heading,Category,Date,Time,URL
0,Times of India,Mumbai may face 10-15% water cut till June 14,mumbai news,2019-06-12,23:18,https://timesofindia.indiatimes.com/city/mumba...
1,Times of India,Gang busted for seizing man's wealth; 13 held,mumbai news,2019-06-12,23:18,https://timesofindia.indiatimes.com/city/mumba...
2,Times of India,Salla's hijack note: Pilot reveals tense moments,mumbai news,2019-06-12,23:18,https://timesofindia.indiatimes.comhttps://mum...
3,Times of India,Hookah ban: Bombay HC admits clutch of pleas,mumbai news,2019-06-12,23:18,https://timesofindia.indiatimes.com/city/mumba...
4,Times of India,Mumbai: Commuters face heavy traffic snarls as...,mumbai news,2019-06-12,23:18,https://timesofindia.indiatimes.com/videos/cit...


In [22]:
outname ='TOI_cities-'+str(Today_date)+'.csv'

root = 'CSV/'
if not os.path.exists(root):
    os.mkdir(root)
date_today= Today_date +'/'
outdir=root+ date_today[:-1]
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)

In [23]:
df.to_csv(fullname,index=False,encoding='utf-8')
